# Equivariance Tutorial

In [ ]:
import torch
from torch.nn.functional import affine_grid, grid_sample
from torchvision.io import read_image
import lietorch
import matplotlib.pyplot as plt

In [ ]:
shoe = read_image("images/shoe.png")[None, ...] / 255.
dress = read_image("images/dress.png")[None, ...] / 255.

In [ ]:
def transform(image, x, A):
    affine_matrix = torch.hstack((A, x[None, ...].T))
    grid = affine_grid(affine_matrix[None, ...], image.shape, align_corners=False)
    return grid_sample(image, grid, align_corners=False)

## Theory

### Lie Groups

> **Definition (Lie Group)** $G$ is a _Lie group_ if it is 
> 1. a _smooth manifold_ - so smooth and looks locally like $\mathbb{R}^n$ - and
> 2. a _group_ - we have a smooth, well-behaved product $\cdot: G \times G \to G$.

The most important Lie groups (imo) encode symmetries on other spaces. 

> **Example (Translation Group)**
> The $n$-dimensional _translation group_ $\mathbb{R}^n$ acts on Euclidean space $\mathbb{R}^n$ by translation, namely
> $$ (\mathbf{x}, \mathbf{y}) \mapsto \mathbf{x} + \mathbf{y}, $$
> and has group product
> $$ (\mathbf{x}, \mathbf{y}) \mapsto \mathbf{x} + \mathbf{y}. $$
Of course, this is an incredibly boring example. 
Slightly less trivial is the following:
> **Example (Special Orthogonal Group)**
> The _special orthogonal group_ $\operatorname{SO}(n)$ acts on Euclidean space $\mathbb{R}^n$ by rotation, namely
> $$ (R, \mathbf{y}) \mapsto R\mathbf{y}, $$
> and has group product
> $$ (R, S) \mapsto RS. $$
Here, we represent the elements of $\operatorname{SO}(n)$ as $n \times n$ orthogonal matrices with determinant $1$. 
Note that in this example the group and the space that is acted on can no longer be identified.

We get our favourite group by combining the two previous ones:
> **Example (Special Euclidean Group)**
> The _special Euclidean group_ $\operatorname{SE}(n)$ acts on Euclidean space $\mathbb{R}^n$ by roto-translation, namely
> $$ ((\mathbf{x}, R), \mathbf{y}) \mapsto \mathbf{x} + R\mathbf{y}, $$
> and has group product
> $$ ((\mathbf{x}, R), (\mathbf{y}, S)) \mapsto (\mathbf{x} + R\mathbf{y}, RS). $$

Many problems have inherent symmetries. For example, . Lie groups give us a mathematically formal way of thinking about these symmetries.

### Equivariance

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10, 5))
ax[0].imshow(shoe.squeeze())
ax[1].imshow(dress.squeeze());

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(15, 5))
x = torch.tensor([0., 0.])
R = torch.tensor([[1., 0.], [0., 1.]])
ax[0].imshow(transform(shoe, x, R).squeeze())

x = torch.tensor([0., 0.])
R = torch.tensor([[0., -1.], [1., 0.]])
ax[1].imshow(transform(shoe, x, R).squeeze())

x = torch.tensor([0., 0.5])
R = torch.tensor([[0., -1.], [1., 0.]])
ax[2].imshow(transform(shoe, x, R).squeeze());

### Lifting

## Application